# ____________________ Sign Language Detection ____________________

# Install and import dependancies

In [1]:

!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -r requirements.txt  # install

!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

c:\Users\xxram\OneDrive\Desktop\Project\gesture_recognition\yolov5


fatal: Too many arguments.

usage: git clone [<options>] [--] <repo> [<dir>]

    -v, --verbose         be more verbose
    -q, --quiet           be more quiet
    --progress            force progress reporting
    --reject-shallow      don't clone shallow repository
    -n, --no-checkout     don't create a checkout
    --bare                create a bare repository
    --mirror              create a mirror repository (implies bare)
    -l, --local           to clone from a local repository
    --no-hardlinks        don't use local hardlinks, always copy
    -s, --shared          setup as shared repository
    --recurse-submodules[=<pathspec>]
                          initialize submodules in the clone
    --recursive ...       alias of --recurse-submodules
    -j, --jobs <n>        number of submodules cloned in parallel
    --template <template-directory>
                          directory from which templates will be used
    --reference <repo>    reference repository
    --refere

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117


In [2]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
from PIL import Image
from PIL import ImageTk
import threading
import tkinter as tk
import pandas as pd
import random

# Import Roboflow for manual dataset

In [ ]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="UxYM4k1Bj7JLXRh9oXIu")
# project = rf.workspace("handrecog").project("hand_images")
# dataset = project.version(1).download("yolov5")

In [ ]:
# from roboflow import Roboflow
# rf = Roboflow(model_format="yolov5", notebook="ultralytics")

# rf = Roboflow(api_key="Sbc4IW9leBI9v22H2p11")
# project = rf.workspace("signlanguage-7gl6w").project("letter_images")
# dataset = project.version(2).download("yolov5")

# Train and validate the model

In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# C:\Users\xxram\OneDrive\Desktop\Project\gesture_recognition\yolov5\Letter_images-2

!python train.py --img 416 --batch 32 --epoch 150 --data sign_language_dataset.yaml --weights yolov5m.pt --cache

In [ ]:
#   Start Time: 10:06
#   End Time: 

In [14]:
!python detect.py --weights runs/train/exp44/weights/best.pt --img 416 --conf 0.3 --source letter_images-2/valid/images

detect: weights=['runs/train/exp44/weights/best.pt'], source=letter_images-2/valid/images, data=data\coco128.yaml, imgsz=[416, 416], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v6.2-267-gbe348cc Python-3.8.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)

Fusing layers... 
Model summary: 212 layers, 20945877 parameters, 0 gradients, 48.2 GFLOPs
image 1/43 C:\Users\xxram\OneDrive\Desktop\Project\gesture_recognition\yolov5\Letter_images-2\valid\images\a_003_jpg.rf.72c0db7d801527c0a5d9fc300e394b99.jpg: 416x416 1 a, 13.0ms
image 2/43 C:\Users\xxram\OneDrive\Desktop\Project\gesture_recognition\yolov5\Letter_images-2\valid\images\b_002_jpg.rf.3863714fa03996a66f4ec630b

In [3]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../yolov5/runs/train/exp44/weights/best.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\xxram/.cache\torch\hub\master.zip
YOLOv5  2023-1-19 Python-3.8.8 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)

Fusing layers... 
Model summary: 212 layers, 20945877 parameters, 0 gradients, 48.2 GFLOPs
Adding AutoShape... 


# Run the model

In [ ]:
cv2.__version__

# Run the below commands in terminal if the there is an error:
#
# pip uninstall opencv-python
# pip install opencv-python

Turns on the camera after the user presses 'Start'. Freezes the frame when the user presses 'Stop'

In [12]:

def random_button_clicked(self):
    global letter
    letter_list = ["a", "b", "c", "d", "e", "f", "g", "i", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
    letter = random.choice(letter_list)


def start_button_clicked(videoloop_stop):
    threading.Thread(target=videoLoop, args=(videoloop_stop,)).start()


def stop_button_clicked(videoloop_stop):
    videoloop_stop[0] = True


def videoLoop(mirror=False):
    cap = cv2.VideoCapture(2)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 800)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 600)

    while True:
        ret, frame = cap.read()
        results = model(frame)

        for i, row in results.pandas().xyxy[0].iterrows():
            name = (row['name'])
            conf = (row['confidence'])
            if conf >= 0.5:
                name_label = tk.Label(root, text=name, bg="green", font=("", 20))
                name_label.place(x=50, y=600, width=400, height=150)
                # print(name, " with confidence: {:.2f}".format(conf))
            elif conf >= 0.3 and conf < 0.5:
                almost_label = tk.Label(root, text = "Nearly There!", bg = "yellow", font=("", 20))
                almost_label.place(x=50, y=600, width=400, height=150)
            elif conf < 0.3:
                low_label = tk.Label(root, text = "Needs Improvement!", bg = "red", font=("", 20))
                low_label.place(x=50, y=600, width=400, height=150)
            
        if mirror is True:
            frame = frame[:, ::-1]
            
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageTk.PhotoImage(image)
        panel = tk.Label(image=image)
        panel.image = image
        panel.place(x=50, y=120)

        if videoloop_stop[0]:
            videoloop_stop[0] = False
            panel.destroy()
            cap.release()
            break

       

videoloop_stop = [False]

root = tk.Tk()
root.title("BSL Learning Tool")
root.geometry("1920x1080+0+0")

letter_list = ["a", "b", "c", "d", "e", "f", "g", "i", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", ]
letter = random.choice(letter_list)

random_button = tk.Button(root, text="New letter", bg="#fff", font=("", 30),command= random_button_clicked)
random_button.place(x=1000, y=100, width=400, height=150)

start_button = tk.Button(root, text="start", bg="#fff", font=("", 50),command=lambda: start_button_clicked(videoloop_stop))
start_button.place(x=1000, y=500, width=400, height=150)

stop_button = tk.Button(root, text="stop", bg="#fff", font=("", 50),command=lambda: stop_button_clicked(videoloop_stop))
stop_button.place(x=1000, y=650, width=400, height=150)

prompt_label = tk.Label(root, text=("Sign", letter), font=("", 30))
prompt_label.place(x=50, y=50)

root.mainloop()